In [ ]:
# export
from fastai_local.core import *
from fastai_local.test import *
from fastai_local.imports import *

In [ ]:
# default_exp notebook.export2html

# Converting notebooks to html

> The functions that transform the dev notebooks in the documentation of the library

## Preprocessing notebook

In [ ]:
#export
def remove_code_cell_jupyter_widget_state_elem(cells):
    widget_str = 'application/vnd.jupyter.widget-view+json'
    for c in cells:
        if c['cell_type'] == 'code':
            if 'outputs' in c:
                c['outputs'] = [l for l in c['outputs'] if not ('data' in l and widget_str in l.data)]
    return cells

In [ ]:
#export
def hide_export_cell(cells):
    

In [ ]:
def remove_undoc_cells(cells):
    old, _, _ = parse_sections(cells)
    return old

# currently code vbox sub-cells mainly
def remove_code_cell_jupyter_widget_state_elem(cells):
    for c in cells:
        if c['cell_type'] == 'code':
            if 'outputs' in c:
                c['outputs'] = [l for l in c['outputs'] if not ('data' in l and 'application/vnd.jupyter.widget-view+json' in l.data)]
    return cells